
## MNIST CNN


In [1]:

## !pip install torchvision


In [2]:

import torch
import numpy as np
import os

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt

from PIL import Image


In [3]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score


In [4]:

import torch.optim as optim 
import torch.nn as nn



## Parameters


In [5]:

learning_rate = 0.003
N_Epochs      = 20
batch_size    = 32



## Get the data


In [6]:

data_path = "data/MNISTdata/"


mnist_train=datasets.MNIST(data_path, train=True, download=True)


In [7]:

mnist_test=datasets.MNIST(data_path, train=False, download=True)


In [8]:

mnist_train_tr = datasets.MNIST(data_path, train=True, download=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor()
                                            ]))
mnist_test_tr  = datasets.MNIST(data_path, train=False, download=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor()
                                            ]))



## Display image


In [9]:

Image.fromarray(  mnist_test_tr.data[123].numpy()  ) 
        

In [10]:

## mnist_train_tr.data = mnist_train_tr.data.view(60000, 1, 28, 28)

mnist_train_tr.data.shape


torch.Size([60000, 28, 28])


## Create Data Loaders


In [11]:

mnist_train_tr.data.shape


torch.Size([60000, 28, 28])

In [12]:

mnist_test_tr.data.shape


torch.Size([10000, 28, 28])

In [13]:

batch_size = 32

train_dl  = torch.utils.data.DataLoader(mnist_train_tr, batch_size=batch_size, shuffle=True  ) 
test_dl   = torch.utils.data.DataLoader(mnist_test_tr,  batch_size=10000,      shuffle=False ) 


In [14]:

def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure



## CNN architecture


In [15]:

class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,             ## the comma is needed 
        
    def forward(self, x):
        return x.view(*self.shape)


In [16]:

class Classifier_CNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()
            
        self.model = nn.Sequential(
                
                ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2), 
 
                ## Convolutional Layer 2
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),   
 
                ## feed forward layer 
                nn.Flatten(),
                nn.Linear(800, 512),    ## see how to get 800 below on last cell
                nn.ReLU(),

                nn.Linear(512, 10),
                nn.LogSoftmax(dim=1)
        )
 
 
           
    def forward(self, inputs):
            
        return self.model(inputs)
    
    


In [17]:

def training_loop( N_Epochs, model, loss_fn, opt  ):
    
    losses_list = []
    
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            
            ## print( xb.shape )   ## check this comes out [N, 1, 28, 28]
            ## yb = torch.squeeze(yb, dim=1)
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
       
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)
            losses_list.append(  loss  )
            
    return losses_list




In [36]:

model          = Classifier_CNN()

opt            = torch.optim.Adam(    model.parameters(), lr=learning_rate )

loss_fn        = nn.CrossEntropyLoss( )   

my_losses_list = training_loop(  N_Epochs, model, loss_fn, opt  )


0 loss= tensor(0.0035, grad_fn=<NllLossBackward0>)
1 loss= tensor(0.0191, grad_fn=<NllLossBackward0>)
2 loss= tensor(0.0094, grad_fn=<NllLossBackward0>)
3 loss= tensor(0.0857, grad_fn=<NllLossBackward0>)
4 loss= tensor(0.0012, grad_fn=<NllLossBackward0>)
5 loss= tensor(0.0007, grad_fn=<NllLossBackward0>)
6 loss= tensor(1.3709e-06, grad_fn=<NllLossBackward0>)
7 loss= tensor(0., grad_fn=<NllLossBackward0>)
8 loss= tensor(0.0128, grad_fn=<NllLossBackward0>)
9 loss= tensor(0.0200, grad_fn=<NllLossBackward0>)
10 loss= tensor(0.0070, grad_fn=<NllLossBackward0>)
11 loss= tensor(0.0022, grad_fn=<NllLossBackward0>)
12 loss= tensor(5.8149e-06, grad_fn=<NllLossBackward0>)
13 loss= tensor(2.6077e-08, grad_fn=<NllLossBackward0>)
14 loss= tensor(3.3155e-07, grad_fn=<NllLossBackward0>)
15 loss= tensor(3.7253e-09, grad_fn=<NllLossBackward0>)
16 loss= tensor(6.4162e-05, grad_fn=<NllLossBackward0>)
17 loss= tensor(0.0002, grad_fn=<NllLossBackward0>)
18 loss= tensor(0.2447, grad_fn=<NllLossBackward0>)
19


## Predict and evaluate with trained model on test set


In [37]:

with torch.no_grad():
    for x_real, y_real in test_dl:
        
        y_pred = model(  x_real  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        print_metrics_function(y_real, preds)
   


Accuracy: 0.99
Confusion Matrix:
[[ 974    0    1    2    0    0    2    1    0    0]
 [   0 1130    1    2    0    0    1    1    0    0]
 [   0    1 1024    0    1    0    1    1    4    0]
 [   0    0    2 1000    0    3    0    0    3    2]
 [   0    1    0    0  974    0    1    0    3    3]
 [   0    1    0    7    0  880    1    2    1    0]
 [   4    3    0    0    0    3  945    0    3    0]
 [   0    1   11    1    2    0    0 1011    0    2]
 [   0    0    0    1    0    1    0    2  968    2]
 [   2    1    0    0    8    1    0    8    4  985]]
Precision: 0.989
Recall: 0.989
F1-mesure: 0.989



## Figuring out the convolution maths for your model


In [18]:

N_batches = 1


In [19]:

my_tensor_test = torch.randn(N_batches , 1, 28,  28)

my_tensor_test.shape


torch.Size([1, 1, 28, 28])

In [20]:

maths_conv1 = nn.Conv2d( 1, 16, kernel_size=5, stride=2)

res_conv1 = maths_conv1(my_tensor_test)

res_conv1.shape


torch.Size([1, 16, 12, 12])

In [21]:

maths_conv2 = nn.Conv2d( 16, 32, kernel_size=5, stride=2)
res_conv2 = maths_conv2(res_conv1)
res_conv2.shape


torch.Size([1, 32, 4, 4])

In [22]:

reshape_res = res_conv2.view(   (N_batches , -1)  )
reshape_res.shape


torch.Size([1, 512])


## Use conv2d and maxpool with large dataset 


In [23]:

large_maths_model =  nn.Sequential(
            
              ## Convolution layer 1
              nn.Conv2d(3, 16, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(16),
              nn.MaxPool2d(2, 2),
              nn.Dropout(0.25),
      
              ## Convolution layer2
              nn.Conv2d(16, 32, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(32),
              nn.MaxPool2d(2, 2),
              nn.Dropout(0.25),
        
        )


In [24]:

N_batches = 10


In [25]:

large_my_tensor_test   = torch.randn(N_batches, 3, 256,  256)

large_res_actual_model = large_maths_model(  large_my_tensor_test   )

large_res_actual_model.shape


torch.Size([10, 32, 15, 15])

In [26]:

large_res_actual_model.view(   (N_batches, -1)  ).shape



torch.Size([10, 7200])


## Another example


In [27]:

maths_model =  nn.Sequential(
            
              ## Convolution layer 1
              nn.Conv2d(1, 16, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(16),
              nn.Dropout(0.25),
      
              ## Convolution layer2
              nn.Conv2d(16, 32, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(32),
              nn.Dropout(0.25),
        
        )


In [28]:

N_batches = 10


In [29]:

my_tensor_test   = torch.randn(N_batches, 1, 28,  28)

res_actual_model = maths_model(  my_tensor_test   )

res_actual_model.shape



torch.Size([10, 32, 4, 4])

In [30]:

res_actual_model.view(   (N_batches, -1)  ).shape


torch.Size([10, 512])


## Now, for this problem


In [31]:

N_batches = 10


In [32]:

model_rc = nn.Sequential(
                    
              
                
                ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),     
 
                ## Convolutional Layer
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),     
    
                nn.Flatten()
)


In [33]:

for xb, yb in train_dl:
    
    print( xb.shape )         ## Dataloader converted from [32, 28, 28] to [32, 1, 28, 28]
    break


torch.Size([32, 1, 28, 28])


In [34]:

my_tensor_test   = torch.randn(N_batches, 1, 28,  28)

res_actual_model = model_rc(  my_tensor_test   )

res_actual_model.shape


torch.Size([10, 800])

In [35]:

res_actual_model.view(   (N_batches, -1)  ).shape


torch.Size([10, 800])